## Gerar o dataset com o arquivo de cdr

In [1]:
#transformando o txt em csv
import csv

with open('cdr.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('cdr_gerado.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        #writer.writerow(('ID', 'CALLING_NUM', 'CALLED_NUM', 'START_TIME', 'END_TIME', 'CALL_TYPE', 'CALL_RESULT'))
        writer.writerows(lines)

In [2]:
# transformando csv em dataframe
import pandas as pd 

df = pd.read_csv("cdr_gerado.csv")
#df.describe()
df.head()
#df.dtypes

,ID,CALLING_NUM,CALLED_NUM,START_TIME,END_TIME,CALL_TYPE,CALL_RESULT
0,1df12368-ecd3-47e7-a082-4e187a5f1953,153191121507,419388847707,2016-12-06 14:17:57.851392,2016-12-06 14:33:14.851392,VOICE,ANSWERED
1,3b5ba4b8-7091-4b71-b55f-f9984c8d45d4,419398318311,218493451709,2016-07-14 14:17:57.851392,2016-07-14 14:22:18.851392,VOICE,ANSWERED
2,97331015-1664-4948-b9fc-54b4c31cdb82,211689462628,216883388316,2018-04-17 14:17:57.851392,2018-04-17 14:39:38.851392,VOICE,ANSWERED
3,c6cb448a-4bd1-4ee8-9cd2-d58e9a720819,157589615514,156195624308,2017-05-26 14:17:57.851392,2017-05-26 14:43:07.851392,VOICE,REJECTED
4,a786ed73-f909-405c-8725-e7d9a021ef23,214281846947,313192565449,2018-06-08 14:17:57.852393,2018-06-08 14:24:19.852393,VOICE,REJECTED


In [6]:
#Calculando a duração
#import datetime

df['START_TIME'] =  pd.to_datetime(df['START_TIME'], format='%Y-%m-%d %H:%M:%S') 
df['END_TIME'] =  pd.to_datetime(df['END_TIME'], format='%Y-%m-%d %H:%M:%S') 
df['DURATION'] = (df.END_TIME - df.START_TIME)
df.head()

#df.tim
#df.dtypes

,ID,CALLING_NUM,CALLED_NUM,START_TIME,END_TIME,CALL_TYPE,CALL_RESULT,DURATION
0,1df12368-ecd3-47e7-a082-4e187a5f1953,153191121507,419388847707,2016-12-06 14:17:57.851392,2016-12-06 14:33:14.851392,VOICE,ANSWERED,00:15:17
1,3b5ba4b8-7091-4b71-b55f-f9984c8d45d4,419398318311,218493451709,2016-07-14 14:17:57.851392,2016-07-14 14:22:18.851392,VOICE,ANSWERED,00:04:21
2,97331015-1664-4948-b9fc-54b4c31cdb82,211689462628,216883388316,2018-04-17 14:17:57.851392,2018-04-17 14:39:38.851392,VOICE,ANSWERED,00:21:41
3,c6cb448a-4bd1-4ee8-9cd2-d58e9a720819,157589615514,156195624308,2017-05-26 14:17:57.851392,2017-05-26 14:43:07.851392,VOICE,REJECTED,00:25:10
4,a786ed73-f909-405c-8725-e7d9a021ef23,214281846947,313192565449,2018-06-08 14:17:57.852393,2018-06-08 14:24:19.852393,VOICE,REJECTED,00:06:22


Definindo os planos horários

In [28]:
# Definindo a estrutura basica dos planos... primeiro, o que é normal e reduzido

def charge():
    # Tabela de tipo basica... 
    #<CHARGE_TYPE><START_TIME><END_TIME><VAL_MINUTE>
    import numpy as np
    data = np.array([['charge_type','start_time','end_time','val_minute'],
                ['Normal','09:00:00','13:00:00', 1.23],
                ['Reduzido','13:10:00','20:00:00', 1.10]])
    return data


In [30]:
## Ver o dtype...
charge()

array([['charge_type', 'start_time', 'end_time', 'val_minute'],
       ['Normal', '09:00:00', '13:00:00', '1.23'],
       ['Reduzido', '13:10:00', '20:00:00', '1.1']], dtype='<U11')

In [31]:
# Verificar as chamadas para definir o valor a aplicar
def val_times_in_series(start_time, end_time, duration):
    #Recuperando a matriz
    bands = charge()
    
    #Localizando a faixa/band inicio
    #Vendo se o start é normal se nao é reduzido
    if bands[1,1] < start_time < bands[1,2] :
        charge_type_start = bands[1,0]
        val_time_start = bands[1,3]
        
    else :
        charge_type_start = bands[2,0]
        val_time_start = bands[2,3]
    
    #Vendo se o end é normal se nao é reduzido
    if bands[1,1] < end_time < bands[1,2] :
        charge_type_end = bands[1,0]
        val_time_end = bands[1,3]
        
    else :
        charge_type_end = bands[2,0]
        val_time_end = bands[2,3]
    
    #Testando se a ligacao esta em duas bandas   
    #os casos de começar numa band e terminar em outra 
    if charge_type_start == charge_type_end :
        #mesma band
        val_call = duration * val_minute_start
        print(val_call)
    else :
        # Diferentes
        # Calculando  parte 1
        if charge_type_start == 'Normal':
            # Pegar o inicio da ligação e o fim da band
        
            val_call1 = (start_time - bands[1,2]) * bands[1,3] 
            val_call2 = (start_time - bands[2,2]) * bands[2,3]
            val_call = val_call1 + val_call1
        else :
            # Pegar o inicio da ligação e o fim da band
            val_call1 = (start_time - bands[1,2]) * bands[1,3] 
            val_call2 = (start_time - bands[2,2]) * bands[2,3]
            val_call = val_call1 + val_call1
        # Calculando  parte 2
        # Pegar o fim da ligação e o inicio da band
        
        
        
    
    return

In [38]:
#Chamar a funcao com o dataframe
for index, row in df.iterrows():
    print (row['START_TIME'], row['END_TIME'], row['DURATION'])
    val_times_in_series(row['START_TIME'], row['END_TIME'], row['DURATION'])
    

2016-12-06 14:17:57.851392 2016-12-06 14:33:14.851392 0 days 00:15:17


TypeError: Cannot compare type 'Timestamp' with type 'str_'